In [2]:
from utils.schedule_utils import load_data_split,table_to_str,table_to_str_sql,find_intersection_and_add_row_id,Prepare_Data_for_Operator_Sequence,format_document,batch_rerank_scores,ROLLBACK,merge_clean_and_format_df_dict,retrieve_rows_by_subtables,process_error_analysis_list
import pickle
import numpy as np
import pandas as pd
import os
import json
from FlagEmbedding import FlagReranker
from tqdm import tqdm
from typing import Dict, Any
from utils.evaluator import Evaluator
from utils.prompt_generate import build_wikitq_prompt_from_df,evaluate_predictions,filter_dataframe_from_responses,fix_sql_query,match_subtables,retrieve_rows_by_subtables,build_tab_fact_prompt_from_df
from utils.async_llm import infer_prompts

In [27]:
ALL_LABELS = ['Base', 'Execute_SQL', 'RAG', 'Select_Column', 'Select_Row']
def parse_valid_route(input):
    parse = eval(input)[0]
    eval_parse = eval(parse)
    ## 如果有非法字符
    return [e for e in eval_parse if e in ALL_LABELS]

In [ ]:
result = np.load('datasets')

In [ ]:
## wikitq: 0.491 second/query 
## tab_fact: 0.29 second/query

In [3]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
from time import sleep
import argparse
from vllm.lora.request import LoRARequest
import json
from tqdm import tqdm
import os 

INFO 10-20 02:56:18 [__init__.py:216] Automatically detected platform cuda.


In [4]:
## 参数记录 for wikitq ablation training-free router
N_PARALLEL = 32 ## only for convert_df_type_parallel.py of data preprocess，一次性离线的
embedding_base_model_path = '/data/workspace/yanmy/models/bge-m3'
router_model_path = '/data/workspace/yanmy/HybridRAG/H-STAR/router/bge-m3-finetuned/' ## fine-tuned router model path
check_moedl_path = '/data/workspace/yanmy/HybridRAG/H-STAR/check/output/bge-reranker-v2-m3-finetuned/' ## check model path
tmp_save_path = 'datasets/schedule_test/wikitq_train_free' ## 临时存储路径
dataset_name = 'wikitq'
split = 'test'
tau = 0.82
check_tau = 0.8
ALL_LABELS = [
    'Base', 'Select_Row', 'Select_Column', 'Execute_SQL', 'RAG_20_5', 
]


In [28]:
RAG_20_5 = np.load('datasets/schedule_test/wikitq/Hybrid_Retrieve_output_update.npy',allow_pickle=True).item()
wikitq_df_processed = np.load('datasets/schedule_test/wikitq/wikitq_df_processed.npy',allow_pickle=True).item()
rewrite_query_list = np.load('datasets/schedule_test/wikitq/rewrite_query_4B.npy',allow_pickle=True)
training_free_router = pd.read_csv('../datasets/ablation/wikitq_test_training_free_router_output_4B.csv',index_col=0)
training_free_router = [parse_valid_route(x) for x in training_free_router['predict'].tolist()]

In [79]:
count = []
for index in range(len(dataset)):
    valid_seq = [s for s in training_free_router[index] if s!='Base']
    count.append(len(valid_seq))
# count / len(dataset)
print(f'0 count number is {count.count(0)}, 1 count number is {count.count(1)}, 2 count number is {count.count(2)}, 3 count number is {count.count(3)}')

0 count number is 403, 1 count number is 960, 2 count number is 2126, 3 count number is 855


In [77]:
training_free_router

[['Select_Column', 'Select_Row', 'Execute_SQL'],
 ['Select_Column', 'Select_Row'],
 ['Select_Row', 'Execute_SQL'],
 ['Select_Row', 'Base'],
 ['Select_Column', 'Execute_SQL'],
 ['Select_Row', 'Select_Column'],
 ['Base'],
 ['Select_Row', 'Base'],
 ['Select_Column', 'Select_Row'],
 ['Select_Row', 'Base'],
 ['Select_Column', 'Select_Row', 'Execute_SQL'],
 ['Select_Row', 'Base'],
 ['Select_Column', 'Select_Row', 'Execute_SQL'],
 ['Select_Column', 'Select_Row', 'Execute_SQL'],
 ['Select_Column', 'Select_Row'],
 ['Base', 'Select_Row', 'Execute_SQL'],
 ['Select_Row', 'Base'],
 ['Select_Row', 'Execute_SQL'],
 ['Select_Row'],
 ['Select_Column', 'Execute_SQL'],
 ['Select_Column', 'Select_Row', 'Execute_SQL'],
 ['Select_Column', 'Select_Row'],
 ['Select_Column', 'Execute_SQL'],
 ['Select_Row', 'Execute_SQL'],
 ['Select_Row'],
 ['Select_Column', 'Select_Row', 'Execute_SQL'],
 ['Select_Column', 'Execute_SQL'],
 ['Select_Row', 'Select_Column'],
 ['Base'],
 ['Select_Row', 'Execute_SQL'],
 ['Select_Row

In [54]:
filtered_table_column = np.load('../datasets/pipeline/filtered_tables_col_test.npy',allow_pickle=True).item()
filtered_table_row = np.load('../datasets/pipeline/filtered_tables_row_test.npy',allow_pickle=True).item()
sql_exec_df_output = np.load('../datasets/pipeline/sql_exec_df_output_test.npy',allow_pickle=True).item()
sql_executable_count = np.load('../datasets/pipeline/wikitq_test_sql_executable_count_30B.npy',allow_pickle=True)
processed_table = {} ## processed_table包括了inference LLM后所有结果的聚合,key 与 method_key 保持一致
processed_table['Base'] = wikitq_df_processed
processed_table['Select_Row'] = filtered_table_row
processed_table['Select_Column'] = filtered_table_column
processed_table['RAG'] = RAG_20_5
processed_table['RAG_20_5'] = RAG_20_5
processed_table['Execute_SQL'] = sql_exec_df_output ## SQL聚合结果！
processed_table['Execute_SQL_count'] = sql_executable_count ## SQL单个结果，用作 CrossEncoder 跨模态查询
# RAG_20_5 = np.load('../datasets/pipeline/retrieved_tables_20_5_test.npy',allow_pickle=True).item()
# RAG_10_3 = np.load('../datasets/pipeline/retrieved_tables_10_3_0.5.npy',allow_pickle=True).item()

In [2]:
import pickle
with open('../router/inference_results.pkl', 'rb') as f:
        load_result = pickle.load(f)

In [15]:
np.save('datasets/schedule_test/wikitq_train_free/processed_table.npy',processed_table)

### Pipeline 的过程分为以下调度步骤
- 所有调度需要的文件均放在了 `schedule_test/datasets/wikitq`与`schedule_test/datasets/tab_fact`文件夹里面
1. （独立的）跑下面的指令：`python convert_df_type_parallel.py --dataset_name tab_fact --split test_small --output_path datasets/schedule_test/tab_fact/wikitq_df_processed.npy --num_workers 32`
2. (独立的)读取 `datasets/schedule_test/tab_fact/wikitq_df_processed.npy`,构造 inference_router 的文件，输出到`datasets/schedule_test/tab_fact/router_query.pkl`
3. (独立的)构造 DB
4. (GPU-21GB) inference router model，输出route result 到 `datasets/schedule_test/tab_fact/inference_result.pkl` 通过命令：`python inference_router.py --input_path datasets/schedule_test/tab_fact/router_query.pkl --model_path /data/workspace/yanmy/HybridRAG/H-STAR/router/bge-m3-router-tab_fact-hn --output_path datasets/schedule_test/tab_fact/inference_result.pkl`
5. (CPU) 解析 router 结果，生成初步的路径并开始任务调度(重排)
6. (GPU-1.8G) 根据 `datasets/schedule_test/tab_fact/RAG_index.npy`，执行 RAG 任务，根据下面的指令
7. (GPU-vLLM) LLM 根据初步路径，对`Select_Row`, `Select_Column`, `Execute_SQL`执行中间命令
8. (CPU) 根据中间结果，link db 做sql command parse，执行 sql,返回 subtable_list
9. (GPU-1.8G) 加载并运行 Check Model，跑 3 次（FORWARD 机制）
10. (GPU-vLLM) add execute SQL
11. (GPU-vLLM) generate qa command, inference vLLM, evaluate


In [16]:
### 读取必要文件（step 2）
dataset = load_data_split(dataset_name,split)
# wikitq_df_processed = np.load(f'{tmp_save_path}/wikitq_df_processed.npy',allow_pickle=True).item()


In [17]:
## step 3: Consturct DB
from utils.multi_db_v2 import NeuralDB, Executor
table_titles = [dataset[i]['table']['page_title'] for i in range(len(dataset))]
db = NeuralDB(tables=wikitq_df_processed, table_titles=table_titles)
executor = Executor()

Initializing NeuralDB...


Creating tables in DB:   0%|          | 0/4344 [00:00<?, ?it/s]

In [72]:
## step 5
# with open(f'{tmp_save_path}/inference_result.pkl', 'rb') as f:
#     error_analysis_row = pickle.load(f)
with open(f'../router/inference_results.pkl', 'rb') as f:
    error_analysis_row = pickle.load(f)
ranked_result = process_error_analysis_list(error_analysis_row, truncate=True, tau=0.5)
pd.DataFrame([str(r) for r in ranked_result.values()]).value_counts()


0                                             
['Select_Row', 'Select_Column', 'Execute_SQL']    1735
['Base']                                           927
['Select_Column', 'Select_Row', 'Execute_SQL']     783
['RAG_20_5', 'Select_Row', 'Execute_SQL']          348
['Select_Row', 'RAG_20_5', 'Execute_SQL']          309
['Select_Column', 'RAG_20_5', 'Execute_SQL']       123
['RAG_20_5', 'Select_Column', 'Execute_SQL']       119
Name: count, dtype: int64

In [29]:
## training-free router
ranked_result = {}
for index in range(len(dataset)):
    ranked_result[index] = training_free_router[index]

In [9]:
## load Hybrid_Retrieve 的结果
RAG_20_5 = np.load(f'{tmp_save_path}/Hybrid_Retrieve_output.npy',allow_pickle=True).item()

- row_sql

- col_sql

- SQL-exec

- Run Check Model

In [68]:
def Prepare_Data_for_Operator_Sequence(index,sequence,dataset, processed_table, remove_operator = []):
    data_entry = {
    'id': dataset[index]['id'],
    'query': dataset[index]['question'], ## 没有ReWrite，暂时不做变化
    # 'table': wikitq_df_processed[index],
    'title': dataset[index]['table']['page_title'], ## 不变
    # 'SQL': sql_command['sql'] + table_to_str(sql_command['table']) ,
        }
    sequence = [s for s in sequence if s not in remove_operator]
    if sequence.__contains__('Execute_SQL'): ## 有Execute_SQL计算符
        sql_list = [sql_count for sql_count in processed_table['Execute_SQL_count'] if sql_count['id']==index]
        # sql_command = sql_list[0]
        if len(sql_list)==0:
            SQL = ''
        else:
            sql_command = sql_list[0]
            SQL = sql_command['sql'] + table_to_str(sql_command['table'])
        data_entry['SQL'] = SQL
    else:
        data_entry['SQL'] = ''
    extraction_sequence = [s for s in sequence if s!='Execute_SQL']
    # print(extraction_sequence)
    if len(extraction_sequence)==1: ## Only One Operator
        method = extraction_sequence[0]
        data_entry['table'] = processed_table[method][index]
    elif len(extraction_sequence)==2:
        table_1 = processed_table[extraction_sequence[0]][index]
        table_2 = processed_table[extraction_sequence[1]][index]
        # print(table_1.shape, table_2.shape)
        table_intersection = find_intersection_and_add_row_id(table_1,table_2)
        data_entry['table'] = table_intersection
    elif len(extraction_sequence)==3:
        table_1 = processed_table[extraction_sequence[0]][index]
        table_2 = processed_table[extraction_sequence[1]][index]
        table_3 = processed_table[extraction_sequence[2]][index]
        # print(table_1.shape, table_2.shape)
        table_intersection = find_intersection_and_add_row_id(table_1,table_2)
        table_intersection = find_intersection_and_add_row_id(table_intersection,table_3)
        data_entry['table'] = table_intersection
    elif len(extraction_sequence)==0:
        data_entry['table'] = processed_table['Base'][index]
    assert data_entry.__contains__('SQL')
    assert data_entry.__contains__('table')
    return data_entry

In [73]:
## Step 9: generated check model data 
# 对每一条数据，维护一个 Sequence_Operator
# 每条数据应该对应一个 Dict,这个Dict 包含了3 个参数：
    # id: index
    # Sequence: Operator Sequence
    # Terminated: True/False
    # Check Score: CrossEncoder 分数
    # Check Status: True/False,当前是否被计算
# 对每条数据，如果 Terminated = True，则不参与计算
# 对应的，如果 Terminated = False，那么查看 Check Status,如果 Statue=False，那么计算 Check_Score
# 每一轮计算之后，Check_Score 更新，Check Statue更新，该轮开始判定，用一个 threshold
# 如果 Check Status = True, Check Score >= Threshold，那么标注 Terminated = True，当前路径被接收
# 如果 Check Status = True, Check Score < Threshold，那么Terminated = False, Check Status = False, Check Score 清空
# Check Score 触发 RollBack 机制，即除了‘Execute_SQL'外，Extract 回退，重新计算当前流程
# 如果当前操作符只有'Execute_SQL’和空（所有 RollBack 均未被接收），那么触发 FORWARD,Terminated  = True，强制修改 Operator Sequence 为['Execute_SQL']
Check_Model_Data_Sequence = {}
for key in ranked_result.keys(): ## 遍历所有数据,初始化
    start_sequence = ranked_result[key]
    Check_Model_Data_Sequence[key] = {}
    Check_Model_Data_Sequence[key]['id'] = key
    Check_Model_Data_Sequence[key]['Sequence'] = start_sequence
    if start_sequence == ['Base'] or start_sequence == ['Execute_SQL']: ## Terminated 不参与计算
        Check_Model_Data_Sequence[key]['Terminated'] = True
        Check_Model_Data_Sequence[key]['Check_Status'] = False
        Check_Model_Data_Sequence[key]['Check_Score'] = 0.0
    else:
        Check_Model_Data_Sequence[key]['Terminated'] = False
        Check_Model_Data_Sequence[key]['Check_Status'] = False
        Check_Model_Data_Sequence[key]['Check_Score'] = 0.0
# Check_Model_Data_Sequence[4]
for key in Check_Model_Data_Sequence.keys():
    data_entry = Prepare_Data_for_Operator_Sequence(key,Check_Model_Data_Sequence[key]['Sequence'],dataset, processed_table)
    # data_entry = Prepare_Data_for_Operator_Sequence(key,Check_Model_Data_Sequence[key]['Sequence'],dataset, processed_table, remove_operator=['Select_Column','Select_Row'])
    # data_entry = Prepare_Data_for_Operator_Sequence(key,Check_Model_Data_Sequence[key]['Sequence'],dataset, processed_table, remove_operator=['Execute_SQL'])
    Check_Model_Data_Sequence[key]['data_entry'] = data_entry
## Check
count_select = 0
count_all = 0
for index in range(len(dataset)):
    df = Check_Model_Data_Sequence[index]['data_entry']['table']
    count_select += df.shape[0] * df.shape[1]
    count_all += wikitq_df_processed[index].shape[0] * wikitq_df_processed[index].shape[1]
count_select / len(dataset)
    

61.148480662983424

In [70]:
np.save('datasets/schedule_test/wikitq/Check_Model_Data_Sequence_wo_check.npy',Check_Model_Data_Sequence)

136.12292817679557

- load ReRanking Model

In [32]:
## step 9 load rerank model
reranker_model = FlagReranker(check_moedl_path, use_fp16=True)

In [74]:
## step 9: iterative check
check_tau = check_tau
for loop in range(3):
    updated_data = batch_rerank_scores(reranker_model, Check_Model_Data_Sequence, batch_size=16)
    Check_Model_Data_Sequence = updated_data
    ## Check Terminal Status
    for key in Check_Model_Data_Sequence.keys():
        if Check_Model_Data_Sequence[key]['Terminated'] == True:
            continue
        else:
            if Check_Model_Data_Sequence[key]['Check_Status'] == True:
                if Check_Model_Data_Sequence[key]['Check_Score'] >= check_tau: ## 0.5 threshold
                    Check_Model_Data_Sequence[key]['Terminated'] = True
                else:
                    Check_Model_Data_Sequence[key]['Terminated'] = False
                    Check_Model_Data_Sequence[key]['Check_Status'] = False
                    Check_Model_Data_Sequence[key]['Check_Score'] = 0.0
                    current_sequence = Check_Model_Data_Sequence[key]['Sequence']
                    ROLLBACK_seq, terminated_flag = ROLLBACK(current_sequence)
                    Check_Model_Data_Sequence[key]['Sequence'] = ROLLBACK_seq
                    Check_Model_Data_Sequence[key]['Terminated'] = terminated_flag
                    if terminated_flag == False:
                        data_entry = Prepare_Data_for_Operator_Sequence(key,ROLLBACK_seq,dataset, processed_table)
                        Check_Model_Data_Sequence[key]['data_entry'] = data_entry
            else:
                raise ValueError("Check Status should be True when Terminated is False after reranking.")

Filtering items to rerank (Terminated=False and data_entry exists)...
Computing scores for 3417 items with batch size 16...


Chunks: 100%|██████████| 2/2 [00:06<00:00,  3.20s/it]


Updating scores in the original dictionary...
Reranking complete.
Filtering items to rerank (Terminated=False and data_entry exists)...
Computing scores for 553 items with batch size 16...


Chunks: 100%|██████████| 2/2 [00:01<00:00,  1.76it/s]

Updating scores in the original dictionary...
Reranking complete.
Filtering items to rerank (Terminated=False and data_entry exists)...
No items to process.


### Check Additional Execute SQL and Execute

In [64]:
Check_Model_Data_Sequence[523]['Sequence']

['Execute_SQL']

In [ ]:
## step 11: generate QA result
prompt_list = []
for index in range(len(dataset)):
    sequence = Check_Model_Data_Sequence[index]['Sequence']
    prompt = build_wikitq_prompt_from_df(dataset,Check_Model_Data_Sequence[index]['data_entry']['table'],index,template_path='../prompts/text_reason_wtq.txt',processed=True)
    if sequence==[] or sequence.__contains__('Execute_SQL'):
        # try:
        evidence = table_to_str_sql(processed_table['Execute_SQL'][index])
        prompt = prompt + evidence
        print(f'SQL at index {index}')
    prompt_list.append(prompt)
# wikitq_df['instruction'] = prompt_list

In [76]:
## step 11: execute QA
# qa_final = response_vllm(llm,prompt_list, sample_num=1)
qa_final = infer_prompts(prompt_list,temperature=0, top_p=1, sample_num=1)
wikitq_df = pd.DataFrame(dataset)
wikitq_df['instruction'] = prompt_list
wikitq_df['predict'] = qa_final[0]
## step 12: evaluate
wikitq_df['predict'] = [str(s) for s in qa_final[0]]
acc_all, error_index_all, format_error_index_all = evaluate_predictions(dataset_name, wikitq_df, dataset)

Processed prompts:   0%|          | 0/4344 [00:00<?, ?it/s]

Processed prompts: 100%|██████████| 4344/4344 [08:54<00:00,  8.12it/s, est. speed input=14294.02 toks/s, output=1932.41 toks/s]


  0%|          | 0/4344 [00:00<?, ?it/s]

Correct Samples: 3271; Total Samples: 4344
Accuracy: 75.30


In [7]:
def calculate_weighted_average(data):
    """
    计算加权平均的 input token/s 和 output token/s。
    
    :param data: 一个列表，每个元素是 [time, input token/s, output token/s]。
    :return: 加权平均的 input token/s 和 output token/s。
    """
    total_time = sum(row[0] for row in data)  # 总时间
    if total_time == 0:
        return 0, 0  # 避免除以零
    
    weighted_input = sum(row[0] * row[1] for row in data) / total_time
    weighted_output = sum(row[0] * row[2] for row in data) / total_time
    
    return weighted_input, weighted_output


# 示例数据
input_data = [
    [12, 2563, 611],  # [time, input token/s, output token/s]
    [124, 6952, 1041],
    [107, 6477, 1632],
    [231, 18429, 1659]
]

# 计算加权平均
weighted_input, weighted_output = calculate_weighted_average(input_data)

print(f"加权平均的 input token/s: {weighted_input}")
print(f"加权平均的 output token/s: {weighted_output}")

加权平均的 input token/s: 12326.881856540085
加权平均的 output token/s: 1464.7025316455697


  0%|          | 0/4344 [00:00<?, ?it/s]

Correct Samples: 3217; Total Samples: 4344
Accuracy: 74.06


Correct Samples: 3281; Total Samples: 4344
Accuracy: 75.53 (Training-Free)
Correct Samples: 3217; Total Samples: 4344
Accuracy: 74.06 (wo Check)
Correct Samples: 3395; Total Samples: 4344
Accuracy: 74.38 (wo table extraction)
Correct Samples: 3225; Total Samples: 4344
Accuracy: 74.24 (wo SQL)
Correct Samples: 3312; Total Samples: 4344
Accuracy: 76.24


In [44]:
wikitq_df.to_csv('../datasets/ablation/wikitq_test_training_free_router_qa_4B.csv')